1. Embedd data using some algorithm (BERT, ELMo, word2vec, flair)
2. Visualise using t-SNE
3. Make sure to colour-code the sentences respectively with PD/HC to visualise the distinction

---

In [28]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

# Set path to find modelling tools for later use
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))


from pandas import read_csv, DataFrame
from pathlib import Path
from haberrspd.embeddings import get_static_sentence_embeddings
import torch
print(torch.cuda.is_available())

# Depending on where I am, set the path
import socket
if socket.gethostname() == 'pax':
    # Monster machine
    data_root = '../data/MJFF/' # My local path
    data_root = Path(data_root)
else:
    # Laptop
    data_root = '/home/nd/data/liverpool/MJFF' # My local path
    data_root = Path(data_root)

# hello data
df = read_csv(data_root / 'preprocessed_EnglishData.csv')
df.shape

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
False


(1170, 4)

## Get Static Embeddings

In [29]:
X = get_static_sentence_embeddings(df,'glove')

/home/neil/cloud/haberrspd/notebooks/../haberrspd/embeddings.py:59: UserWarning:

This method is currently only designed for English.



Embedding sentence nr: 0
Embedding sentence nr: 100
Embedding sentence nr: 200
Embedding sentence nr: 300
Embedding sentence nr: 400
Embedding sentence nr: 500
Embedding sentence nr: 600
Embedding sentence nr: 700
Embedding sentence nr: 800
Embedding sentence nr: 900
Embedding sentence nr: 1000
Embedding sentence nr: 1100


## Get Dynamic Embeddings

In [2]:
# Init embedding
embedding = ELMoEmbeddings('medium')
sentence_embeddings = DocumentRNNEmbeddings([embedding],bidirectional=False)
# Create a sentence
sentence = Sentence('The grass is green .')
# embed the sentence with our document embedding
sentence_embeddings.embed(sentence)
# now check out the embedded sentence.
print(sentence.get_embedding().shape)

torch.Size([128])


/home/neil/anaconda3/envs/py36/lib/python3.6/site-packages/allennlp/nn/util.py:149: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  index_range = sequence_lengths.new_tensor(torch.arange(0, len(sequence_lengths)))


In [70]:
# Init embedding
embedding = BertEmbeddings('bert-base-cased')
# Create a sentence
sentence = Sentence('The grass is green .')
# Embed words in sentence
embedding.embed(sentence)

100%|██████████| 404400730/404400730 [01:03<00:00, 6323696.47B/s]


[Sentence: "The grass is green ." - 5 Tokens]

In [47]:
# Init embedding
embedding = FlairEmbeddings('news-forward')
# Create a sentence
sentence = Sentence(df['Preprocessed_typed_sentence'][0])
# Embed words in sentence
embedding.embed(sentence)

[Sentence: "Although early behavioural or cognitive intervention can help children gain self-care, social, and communication skills, their is no known cure" - 20 Tokens]

In [4]:
# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = [WordEmbeddings('glove'), 
                      FlairEmbeddings('news-forward')]
sentence_embeddings = DocumentRNNEmbeddings(stacked_embeddings,
                                            hidden_size=32,
                                            bidirectional=False,
                                            rnn_type='LSTM')

## Visualisation

In [31]:
from haberrspd.plotting import sentence_embeddings_scatter_3d_plot

### Load reference sentences

In [11]:
read_csv(data_root / 'EnglishData_target_sentences.csv')

,sentence_id,sentence_text
0,55,"However, religions other than Islam, use a dif..."
1,56,"He is buried in Egypt, Aswan at the Mausoleum ..."
2,57,"Books include Penguin Island, a satire on the ..."
3,58,The w-shaped glyph above the second consonant ...
4,59,The Franks alliance was important exactly beca...
5,60,"As of 2004, nearly 50% of Americans who were e..."
6,61,Lincoln's coffin would be encased in concrete ...
7,62,"Generally considered a part of Central Asia, i..."
8,63,Although early behavioural or cognitive interv...
9,64,The Korean men have not fared so well in Olymp...


In [45]:
sentence_embeddings_scatter_3d_plot(df,X,62,save_me=True,filename='sent_62_glove_')